In [20]:
using Turf
using Shapefile
using CSV
using Glob
using DataFrames
using Missings
using ProgressBars

In [3]:
# confirm that sensor numbers/locations did not change year-to-year
sensor_meta = vcat(map(glob("../data/d*meta*.tsv")) do fn
            return CSV.read(fn, DataFrame, delim="\t")
    end...)

┌ Warning: thread = 2 warning: parsed expected 18 columns, but didn't reach end of line around data row: 970. Ignoring any extra columns on this row
└ @ CSV /Users/matthewc/.julia/packages/CSV/Zl2ww/src/file.jl:613
┌ Warning: thread = 2 warning: parsed expected 18 columns, but didn't reach end of line around data row: 974. Ignoring any extra columns on this row
└ @ CSV /Users/matthewc/.julia/packages/CSV/Zl2ww/src/file.jl:613
┌ Warning: thread = 2 warning: parsed expected 18 columns, but didn't reach end of line around data row: 1265. Ignoring any extra columns on this row
└ @ CSV /Users/matthewc/.julia/packages/CSV/Zl2ww/src/file.jl:613
┌ Warning: thread = 2 warning: parsed expected 18 columns, but didn't reach end of line around data row: 1266. Ignoring any extra columns on this row
└ @ CSV /Users/matthewc/.julia/packages/CSV/Zl2ww/src/file.jl:613
┌ Warning: thread = 2 warning: parsed expected 18 columns, but didn't reach end of line around data row: 1369. Ignoring any extra columns 

,ID,Fwy,Dir,District,County,City,State_PM,Abs_PM,Latitude,Longitude
,Int64,Int64,String,Int64,Int64,Int64?,Any,Float64,Float64?,Float64?
1,308511,50,E,3,17,missing,31.627,60.162,38.7611,-120.57
2,308512,50,W,3,17,missing,31.627,60.166,38.7612,-120.57
3,311831,5,S,3,67,missing,10.896,506.189,38.4098,-121.484
4,311832,5,S,3,67,missing,10.896,506.189,38.4098,-121.484
5,311844,5,N,3,67,missing,11.105,506.398,38.4128,-121.484
6,311847,5,N,3,67,missing,12.185,507.478,38.4283,-121.488
7,311864,5,N,3,67,missing,11.933,507.226,38.4246,-121.487
8,311903,50,E,3,67,64000,L0.633,3.789,38.5669,-121.506
9,311930,50,E,3,67,64000,L0.632,3.788,38.5669,-121.506


In [4]:
uzas = Shapefile.Table("../data/tl_2020_us_uac10.shp")

Shapefile.Table{Union{Missing, Shapefile.Polygon}} with 3601 rows and the following 13 columns:
	
geometry, UACE10, GEOID10, NAME10, NAMELSAD10, LSAD10, MTFCC10, UATYP10, FUNCSTAT10, ALAND10, AWATER10, INTPTLAT10, INTPTLON10


In [9]:
sensor_meta.geom = passmissing(Point).(sensor_meta.Longitude, sensor_meta.Latitude)

18660-element Vector{Union{Missing, Point}}:
 Point([-120.569835, 38.761062])
 Point([-120.569866, 38.761182])
 Point([-121.48412, 38.409782])
 Point([-121.48412, 38.409782])
 Point([-121.484363, 38.412779])
 Point([-121.487578, 38.428258])
 Point([-121.486808, 38.424648])
 Point([-121.505888, 38.566906])
 Point([-121.505906, 38.566911])
 Point([-121.495585, 38.564153])
 Point([-121.495585, 38.564153])
 Point([-121.495585, 38.564153])
 Point([-121.493316, 38.563816])
 ⋮
 Point([-117.672686, 33.548803])
 Point([-117.672993, 33.548807])
 Point([-117.674195, 33.541593])
 Point([-117.6736, 33.541637])
 Point([-117.666416, 33.521679])
 Point([-117.672848, 33.558096])
 Point([-117.672778, 33.560694])
 Point([-117.673104, 33.56069])
 Point([-117.674195, 33.541593])
 Point([-117.6736, 33.541637])
 Point([-117.672848, 33.558096])
 Point([-117.673168, 33.558094])

In [17]:
uza_geom = GeoInterface.MultiPolygon.(GeoInterface.coordinates.(Shapefile.shapes(uzas)))

3601-element Vector{MultiPolygon}:
 MultiPolygon([[[[-89.498589, 41.854668], [-89.498578, 41.855118], [-89.497935, 41.855118999999995], [-89.496522, 41.855118999999995], [-89.496521, 41.855177999999995], [-89.49651899999999, 41.855214], [-89.496516, 41.855320999999996], [-89.496516, 41.855357999999995], [-89.496515, 41.855395], [-89.497928, 41.855382]  …  [-89.501025, 41.852807999999996], [-89.501021, 41.852987], [-89.50101699999999, 41.853165], [-89.501013, 41.853342999999995], [-89.501009, 41.853522], [-89.501004, 41.853739999999995], [-89.500998, 41.854001], [-89.50099399999999, 41.85422], [-89.49860000000001, 41.85422], [-89.498589, 41.854668]]]])
 MultiPolygon([[[[-87.095185, 45.796921999999995], [-87.094961, 45.796921999999995], [-87.094623, 45.796923], [-87.09428899999999, 45.796923], [-87.094066, 45.796923], [-87.09312899999999, 45.796924], [-87.092885, 45.796924999999995], [-87.09127199999999, 45.796904999999995], [-87.090475, 45.796901], [-87.09031399999999, 45.79690499999999

In [27]:
# filter to just California
ca_uza_geom = uza_geom[occursin.(["CA"], uzas.NAME10)]

211-element Vector{MultiPolygon}:
 MultiPolygon([[[[-119.86913200000001, 36.430831999999995], [-119.870931, 36.430828999999996], [-119.87172699999999, 36.430828], [-119.872091, 36.430825], [-119.87355600000001, 36.430813], [-119.87633, 36.43081], [-119.877937, 36.430808999999996], [-119.87813, 36.430800000000005], [-119.87813, 36.432176], [-119.878131, 36.432255]  …  [-119.86461299999999, 36.429871], [-119.864623, 36.430048], [-119.864625, 36.430571], [-119.864626, 36.430805], [-119.865527, 36.430808999999996], [-119.86635799999999, 36.430814], [-119.867269, 36.430814], [-119.86823, 36.430811], [-119.86913200000001, 36.430808999999996], [-119.86913200000001, 36.430831999999995]]]])
 MultiPolygon([[[[-119.144544, 36.306106], [-119.143445, 36.306114], [-119.142922, 36.306118], [-119.140986, 36.306036999999996], [-119.140151, 36.306003], [-119.139054, 36.305958], [-119.138358, 36.305952999999995], [-119.13627199999999, 36.305941], [-119.135577, 36.305937], [-119.13558499999999, 36.3057949

In [33]:
sensor_meta.urban = map(sensor_meta.geom) do g
    if ismissing(g)
        return missing
    else
        for poly in ca_uza_geom
            if point_in_polygon(g, poly)
                return true
            end
        end
    end
    
    return false  # not in any UZA
end

18660-element Vector{Union{Missing, Bool}}:
 false
  true
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
     ⋮
  true
  true
  true
  true
  true
  true
  true
  true
  true
  true
  true
  true

In [34]:
sum(skipmissing(sensor_meta.urban))

7257

In [35]:
CSV.write("../data/sensor_meta_geo.csv", sensor_meta)

"../data/sensor_meta_geo.csv"